In [7]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from IPython.core.display import HTML
import datetime
import numpy as np
import numpy.ma as ma
import os

In [8]:
year = 18; w = 470
#year = 19; w = 543

In [9]:
save = os.getcwd()
save = os.path.join(save,"save")
save = os.path.join(save,str(year))
save = os.path.join(save,'df_rich.pickle')

with open(save, 'rb') as f:
    df = pickle.load(f)

In [10]:
df.rename(columns={"position":"Position"})

,Name,club,Position,Played,Goals,Assists,Clean Sheets,Conceded,Points,Points Last Season,...,Highest R2 Bid,Round 2 Activty,Round 3 Average,Round 3 Bids,Highest R3 Bid,Round 3 Activty,Round 4 Average,Round 4 Bids,Highest R4 Bid,Round 4 Activty
0,Petr Cech,Arsenal,Goalkeeper,6.0,0.0,0.0,1.0,9.0,5.0,42.0,...,11.0,True,5.800000,5,11.0,True,0.000000,0,0.0,False
1,Bernd Leno,Arsenal,Goalkeeper,32.0,0.0,0.0,6.0,42.0,34.0,NaN,...,10.5,True,5.875000,4,10.0,True,0.000000,0,0.0,False
2,David Ospina,Arsenal,Goalkeeper,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,False,1.000000,1,1.0,True,0.000000,0,0.0,False
3,D Button,Brighton & Hove Albion,Goalkeeper,4.0,0.0,0.0,1.0,5.0,5.0,NaN,...,0.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
4,M Ryan,Brighton & Hove Albion,Goalkeeper,34.0,0.0,0.0,6.0,55.0,25.0,42.0,...,5.0,True,12.000000,1,12.0,True,0.000000,0,0.0,False
5,J Steele,Brighton & Hove Albion,Goalkeeper,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,False,0.000000,0,0.0,False,0.000000,0,0.0,False
6,Asmir Begovic,Bournemouth,Goalkeeper,24.0,0.0,0.0,5.0,47.0,11.0,27.0,...,2.0,True,0.000000,1,0.0,True,0.000000,1,0.0,True
7,Artur Boruc,Bournemouth,Goalkeeper,12.0,0.0,0.0,4.0,18.0,14.0,0.0,...,0.0,False,0.000000,0,0.0,False,0.000000,0,0.0,False
8,Tom Heaton,Burnley,Goalkeeper,19.0,0.0,0.0,4.0,27.0,19.0,2.0,...,8.5,True,2.600000,5,5.5,True,0.000000,0,0.0,False
9,Nick Pope,Burnley,Goalkeeper,0.0,0.0,0.0,0.0,0.0,0.0,57.0,...,7.0,True,1.500000,2,3.0,True,0.000000,0,0.0,False


 ## Naive Dynamic Programming

In [11]:
items = df[['Name','Highest R1 Bid','Points']]
items.Points = items.Points.astype(int)
items['Highest R1 Bid'] = items['Highest R1 Bid'].astype(int)*2

C:\PhD\Anaconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
C:\PhD\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [12]:
def knapsack_pandas(items, limit):
    table = np.zeros((items.shape[0]+1,limit+1),dtype=int)
 
    for j in range(1, items.shape[0] + 1):
        item, wt, val = items.iloc[j-1]
        for w in range(1, limit + 1):
            if wt > w:
                table[j][w] = table[j-1][w]
            else:
                table[j][w] = max(table[j-1][w],
                                  table[j-1][w-wt] + val)
 
    result = pd.DataFrame(columns=['item','weight','value'])
    
    w = limit
    for j in range(len(items), 0, -1):
        was_added = table[j][w] != table[j-1][w]
 
        if was_added:
            item, wt, val = items.iloc[j-1]
            result = result.append(items.iloc[j-1])
            w -= wt
            
    return (result, table) 

In [13]:
(bagged, table) = knapsack_pandas(items, 75)
display(HTML(bagged.to_html()))

,item,weight,value,Highest R1 Bid,Name,Points
545,NaN,NaN,NaN,0.0,Jack Simpson,3.0
544,NaN,NaN,NaN,0.0,Mark Travers,1.0
543,NaN,NaN,NaN,0.0,A Rowe,1.0
542,NaN,NaN,NaN,0.0,Ben Johnson,1.0
541,NaN,NaN,NaN,0.0,Carl Jenkinson,1.0
540,NaN,NaN,NaN,0.0,D Duhaney,2.0
539,NaN,NaN,NaN,0.0,Callum Slattery,1.0
537,NaN,NaN,NaN,0.0,Youri Tielemans,32.0
536,NaN,NaN,NaN,0.0,H Nordtveit,6.0
535,NaN,NaN,NaN,0.0,L Markovic,1.0


## Linear Programming

In [14]:
from scipy.optimize import linprog
w = 543
dfl = df.iloc[0:w]

In [15]:
points = dfl['Points']
cost = dfl['Highest R1 Bid']+0.5
gk   = dfl['Position'].apply(lambda x: 1 if x == 'Goalkeeper' else 0)
defe = dfl['Position'].apply(lambda x: 1 if x == 'Defender' else 0)
mid  = dfl['Position'].apply(lambda x: 1 if x == 'Midfielder' else 0)
stri = dfl['Position'].apply(lambda x: 1 if x == 'Forward' else 0)
xi   = np.ones(dfl.shape[0])

In [16]:
A_upperbounds = np.array([cost])
b_upperbounds = np.array([75])
A_equality = np.array([gk, defe, mid, stri, xi])
b_equality = np.array([1, 4, 4, 2, 11])
bounds = [(0, 1) for x in range(dfl.shape[0])]

In [17]:
solution = linprog(
    c=-points,
    A_ub=A_upperbounds,
    b_ub=b_upperbounds,
    A_eq=A_equality,
    b_eq=b_equality,
    bounds=bounds
)

C:\PhD\Anaconda3\lib\site-packages\scipy\optimize\_linprog_util.py:704: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  warn(redundancy_warning, OptimizeWarning)


In [18]:
dfs = dfl[solution.x>0.001]
dfs

,Name,club,Position,Played,Goals,Assists,Clean Sheets,Conceded,Points,Points Last Season,...,Highest R2 Bid,Round 2 Activty,Round 3 Average,Round 3 Bids,Highest R3 Bid,Round 3 Activty,Round 4 Average,Round 4 Bids,Highest R4 Bid,Round 4 Activty
96,T A-Arnold,Liverpool,Defender,28.0,1.0,13.0,16.0,14.0,103.0,40.0,...,25.0,True,4.166667,6,8.0,True,4.000000,1,4.0,True
100,Andrew Robertson,Liverpool,Defender,36.0,0.0,12.0,21.0,20.0,118.0,57.0,...,13.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
181,V Van Dijk,Liverpool,Defender,38.0,4.0,3.0,20.0,22.0,112.0,38.0,...,15.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
183,Aymeric Laporte,Manchester City,Defender,34.0,3.0,3.0,18.0,21.0,98.0,21.0,...,11.0,True,7.687500,8,16.0,True,4.000000,5,8.0,True
236,David Brooks,Bournemouth,Midfielder,29.0,7.0,6.0,NaN,NaN,62.0,NaN,...,0.0,False,0.000000,0,0.0,False,0.000000,1,0.0,True
238,Ryan Fraser,Bournemouth,Midfielder,36.0,7.0,15.0,NaN,NaN,87.0,44.0,...,3.5,True,2.142857,7,6.5,True,2.357143,7,10.0,True
262,Eden Hazard,Chelsea,Midfielder,32.0,16.0,15.0,NaN,NaN,110.0,74.0,...,14.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
269,Luka Milivojevic,Crystal Palace,Midfielder,38.0,12.0,2.0,NaN,NaN,78.0,69.0,...,14.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
360,Nathan Redmond,Southampton,Midfielder,38.0,6.0,6.0,NaN,NaN,68.0,33.0,...,1.5,True,6.000000,5,11.0,True,1.071429,7,6.5,True
412,Callum Wilson,Bournemouth,Forward,29.0,14.0,12.0,NaN,NaN,95.0,51.0,...,17.0,True,11.666667,3,25.0,True,0.000000,0,0.0,False


In [19]:
solution.x[solution.x>0.001]

array([1.        , 1.        , 1.        , 1.        , 0.26666667,
       1.        , 1.        , 0.73333333, 1.        , 1.        ,
       1.        , 1.        ])

In [20]:
dfs[['Name','Highest R1 Bid']]

,Name,Highest R1 Bid
96,T A-Arnold,6.5
100,Andrew Robertson,13.0
181,V Van Dijk,12.0
183,Aymeric Laporte,1.5
236,David Brooks,0.0
238,Ryan Fraser,2.0
262,Eden Hazard,21.5
269,Luka Milivojevic,7.5
360,Nathan Redmond,0.0
412,Callum Wilson,2.5


## Integrer Programming

In [41]:
from pulp import *
dfi = df.iloc[0:w]

In [42]:
dfi.club.unique()

array(['Arsenal', 'Brighton & Hove Albion', 'Bournemouth', 'Burnley',
       'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield Town', 'Leicester City', 'Liverpool',
       'Manchester City', 'Manchester Utd', 'Newcastle United',
       'Southampton', 'Tottenham', 'Watford', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

In [43]:
player = [dfi['Name'][i] for i in range(dfi.shape[0])]
point = {dfi['Name'][i]: dfi['Points'][i] for i in range(dfi.shape[0])} 
cost = {dfi['Name'][i]: dfi['Highest R1 Bid'][i] + 0.5 for i in range(dfi.shape[0])}
gk = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Goalkeeper' else 0 for i in range(dfi.shape[0])}
defe = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Defender' else 0 for i in range(dfi.shape[0])}
mid = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Midfielder' else 0 for i in range(dfi.shape[0])}
stri = {dfi['Name'][i]: 1 if dfi['Position'][i] == 'Forward' else 0 for i in range(dfi.shape[0])}
city = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Manchester City' else 0 for i in range(dfi.shape[0])}
pool = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Liverpool' else 0 for i in range(dfi.shape[0])}
chel = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Chelsea' else 0 for i in range(dfi.shape[0])}
arse = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Arsenal' else 0 for i in range(dfi.shape[0])}
tot = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Tottenham' else 0 for i in range(dfi.shape[0])}
utd = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Manchester Utd' else 0 for i in range(dfi.shape[0])}
bmo = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Bournemouth' else 0 for i in range(dfi.shape[0])}
lei = {dfi['Name'][i]: 1 if dfi['club'][i] == 'Leicester City' else 0 for i in range(dfi.shape[0])}
xi = {dfi['Name'][i]: 1 for i in range(dfi.shape[0])}

In [44]:
prob = LpProblem("Fantasy Football",LpMaximize)
player_vars = LpVariable.dicts("Players",player,0,1,LpBinary)

# objective function
prob += lpSum([point[i]*player_vars[i] for i in player]), "Total Cost"

# constraint
prob += lpSum([player_vars[i] for i in player]) == 11, "Total 11 Players"
prob += lpSum([cost[i] * player_vars[i] for i in player]) <= 75.0, "Total Cost"
prob += lpSum([city[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([pool[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([chel[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([arse[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([tot[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([utd[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([bmo[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([lei[i] * player_vars[i] for i in player]) <= 2
prob += lpSum([gk[i] * player_vars[i] for i in player]) == 1, "1 GK"
prob += lpSum([defe[i] * player_vars[i] for i in player]) == 4, "4 DEF"
prob += lpSum([mid[i] * player_vars[i] for i in player]) == 4, "4 MID"
prob += lpSum([stri[i] * player_vars[i] for i in player]) == 2, "2 STR"

In [45]:
status = prob.solve()

In [46]:
names = []
for v in prob.variables():
    if v.varValue>0:
        nv = v.name.split("_")[1:]
        if len(nv) > 2:
            ln = "-".join(nv[-2:])
            ln2 = " ".join(nv[-2:])
        else:
            ln = nv[-1]
        fn = nv[0]
        if fn == ln:
            names.append(fn)
        else:
            names.append(fn + " " + ln)
            try:
                names.append(fn + " " + ln2)
            except:
                pass
names

['Andrew Robertson',
 'Andrew A Arnold',
 'Aymeric Laporte',
 'Aymeric A Arnold',
 'Callum Wilson',
 'Callum A Arnold',
 'Ederson',
 'Lucas Digne',
 'Lucas A Arnold',
 'Luka Milivojevic',
 'Luka A Arnold',
 'Nathan Redmond',
 'Nathan A Arnold',
 'Paul Pogba',
 'Paul A Arnold',
 'R Jimenez',
 'R A Arnold',
 'Ryan Fraser',
 'Ryan A Arnold',
 'T A-Arnold',
 'T A Arnold']

In [47]:
df_select = dfi[dfi.Name.isin(names)]
df_select

,Name,club,Position,Played,Goals,Assists,Clean Sheets,Conceded,Points,Points Last Season,...,Highest R2 Bid,Round 2 Activty,Round 3 Average,Round 3 Bids,Highest R3 Bid,Round 3 Activty,Round 4 Average,Round 4 Bids,Highest R4 Bid,Round 4 Activty
29,Ederson,Manchester City,Goalkeeper,38.0,0.0,1.0,20.0,23.0,95.0,76.0,...,27.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
96,T A-Arnold,Liverpool,Defender,28.0,1.0,13.0,16.0,14.0,103.0,40.0,...,25.0,True,4.166667,6,8.0,True,4.000000,1,4.0,True
100,Andrew Robertson,Liverpool,Defender,36.0,0.0,12.0,21.0,20.0,118.0,57.0,...,13.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
183,Aymeric Laporte,Manchester City,Defender,34.0,3.0,3.0,18.0,21.0,98.0,21.0,...,11.0,True,7.687500,8,16.0,True,4.000000,5,8.0,True
238,Ryan Fraser,Bournemouth,Midfielder,36.0,7.0,15.0,NaN,NaN,87.0,44.0,...,3.5,True,2.142857,7,6.5,True,2.357143,7,10.0,True
269,Luka Milivojevic,Crystal Palace,Midfielder,38.0,12.0,2.0,NaN,NaN,78.0,69.0,...,14.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
343,Paul Pogba,Manchester Utd,Midfielder,34.0,13.0,10.0,NaN,NaN,93.0,65.0,...,28.0,True,0.000000,0,0.0,False,0.000000,0,0.0,False
360,Nathan Redmond,Southampton,Midfielder,38.0,6.0,6.0,NaN,NaN,68.0,33.0,...,1.5,True,6.000000,5,11.0,True,1.071429,7,6.5,True
412,Callum Wilson,Bournemouth,Forward,29.0,14.0,12.0,NaN,NaN,95.0,51.0,...,17.0,True,11.666667,3,25.0,True,0.000000,0,0.0,False
469,R Jimenez,Wolverhampton Wanderers,Forward,36.0,13.0,10.0,NaN,NaN,95.0,NaN,...,8.5,True,3.269231,13,13.0,True,1.555556,9,3.5,True


In [48]:
df_select["Bought for"] = df_select['Highest R1 Bid'] + 0.5
df_select[["Name","club","Position","Points","Bought for"]]

C:\PhD\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Name,club,Position,Points,Bought for
29,Ederson,Manchester City,Goalkeeper,95.0,16.5
96,T A-Arnold,Liverpool,Defender,103.0,7.0
100,Andrew Robertson,Liverpool,Defender,118.0,13.5
183,Aymeric Laporte,Manchester City,Defender,98.0,2.0
238,Ryan Fraser,Bournemouth,Midfielder,87.0,2.5
269,Luka Milivojevic,Crystal Palace,Midfielder,78.0,8.0
343,Paul Pogba,Manchester Utd,Midfielder,93.0,15.5
360,Nathan Redmond,Southampton,Midfielder,68.0,0.5
412,Callum Wilson,Bournemouth,Forward,95.0,3.0
469,R Jimenez,Wolverhampton Wanderers,Forward,95.0,5.5


In [49]:
df_select.Points.sum()

1006.0

In [50]:
df_select["Bought for"].sum()

74.5